# Austin Texas Food Inspection

In [1]:
import pandas as pd

# used for getting latitude and longitude from google maps
import re
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver

# evade scraping detection
import time
import random

In [2]:
# import the data
df = pd.read_csv('data/Food_Establishment_Inspection_Scores.csv', parse_dates=['Inspection Date'])

## Clean the Data

In [3]:
# extract coordinates out of the address
df['coordinates'] = df['Address'].str.extract('\(([^)]+)')

In [4]:
# turn coordinates into lists of two values
df['coordinates'] = df['coordinates'].str.split(',')

In [5]:
# extract the lat/long from the coordinates into their own seperate columns
df['original_latitude'] = df['coordinates'].str[0]
df['original_longitude'] = df['coordinates'].str[1]

In [6]:
df['Address'] = df['Address'].str.replace('\(([^)]+)', '', regex=True) # delete longitudes and latitudes out from address
df['Address'] = df['Address'].str.replace('\n', ' ', regex=True) # delete \n
df['Restaurant Name'] = df['Restaurant Name'].str.replace('(#\d+\w)', '', regex=True) # delete odd tags on some names
df['Restaurant Name'] = df['Restaurant Name'].str.replace('\|/', '', regex=True) # delete slashes that interfere with search
df['Address'] = df['Address'].str.strip(')')

In [7]:
df.sort_values(by='Inspection Date', inplace=True, ascending=False)

In [8]:
all_scores = df
df = df.drop_duplicates(subset=['Restaurant Name', 'Address'], keep='first')

In [9]:
df.reset_index(drop=True, inplace=True)
all_scores.reset_index(drop=True, inplace=True)

## Scrape Latitude and Longitude from Google Maps

In [10]:
latitude_list = []
longitude_list = []

In [12]:
driver = webdriver.Chrome()
for row, col in df.iterrows():
    try:
        name_address = col['Restaurant Name'] + ' ' + col['Address']
        driver.get(f'https://www.google.com/maps/search/{name_address}')
        html = driver.page_source
        soup = BeautifulSoup(html)
        time.sleep(random.randint(2, 5))
        match = re.search('@(\d*.\d+,-\d*.\d*)', str(soup))
        coordinates = match[0].strip('@').split(',')
        latitude = coordinates[0]
        longitude = coordinates[1]
        latitude_list.append(latitude)
        longitude_list.append(longitude)
        print(name_address, 'is complete')
    except:
        try:
            driver.get(f'https://www.google.com/maps/search/{col["Address"]}')
            html = driver.page_source
            soup = BeautifulSoup(html)
            time.sleep(random.randint(2, 5))
            match = re.search('@(\d*.\d+,-\d*.\d*)', str(soup))
            coordinates = match[0].strip('@').split(',')
            latitude = coordinates[0]
            longitude = coordinates[1]
            latitude_list.append(latitude)
            longitude_list.append(longitude)
            print(name_address, 'is complete')
        except TypeError as error:
            print('TypeError: ', error, name_address)
            latitude_list.append(col['original_latitude'])
            longitude_list.append(col['original_longitude'])

Comfort Suites 5001 S IH AUSTIN, TX 78744 is complete
Mr. Natural 2414 S LAMAR BLVD AUSTIN, TX 78704  is complete
LW - Dominos Pizza 900 S FM 620 RD LAKEWAY, TX 78734 is complete
Star Stop 79 12801 N FM 620 RD AUSTIN, TX 78750  is complete
Capital One Downtown Austin 106 E 6TH ST AUSTIN, TX 78701  is complete
Papa Murphy's Pizza 2110 W SLAUGHTER LN AUSTIN, TX 78748  is complete
1st Express 4111 S 1ST ST AUSTIN, TX 78745  is complete
Collinfield House, The 9205 COLLINFIELD DR AUSTIN, TX 78758  is complete
7-Eleven Convenience Store  6518 ED BLUESTEIN BLVD AUSTIN, TX 78723  is complete
Prime Food Mart 601 N IH AUSTIN, TX 78702 is complete
Korean Grill 10901 N LAMAR BLVD AUSTIN, TX 78753  is complete
In-N-Out Burger 4515 AIRPORT BLVD AUSTIN, TX 78751  is complete
MN - Las Salsas Bar and Grille 109 N BURNET ST MANOR, TX 78653  is complete
Speedi-Stop 1621 E CESAR CHAVEZ ST AUSTIN, TX 78702  is complete
7 - Eleven Convenience Store  701 W BRAKER LN AUSTIN, TX 78753  is complete
MN - Subway 

Accent Food Services - Rutherford Campus 1520 RUTHERFORD LN AUSTIN, TX 78754  is complete
TypeError:  'NoneType' object is not subscriptable WL - Chick-Fil-A 701 S CAPITAL OF TEXAS HWY NB UNIT L WEST LAKE HILLS TX 78746 
Phil's Ice House 5620 BURNET RD AUSTIN, TX 78756  is complete
Bread Basket, The 1514 HOLLY ST AUSTIN, TX 78702  is complete
Ryan Sanders Sports Services 5400 JIMMY CLAY DR AUSTIN, TX 78744  is complete
Park on South Lamar, The 4024 S LAMAR BLVD AUSTIN, TX 78704  is complete
La Buena Supermercado (Kitchen) 512 W STASSNEY LN AUSTIN, TX 78745  is complete
Wendy's W118 1000 E 41ST ST AUSTIN, TX 78751  is complete
BJ's Restaurant & Brewhouse 10515 N MOPAC EXPY AUSTIN, TX 78759  is complete
Sonic Drive - In  155 W SLAUGHTER LN AUSTIN, TX 78748  is complete
PF - Hilton Home2/Tru Pflugerville 18700 HILLTOP COMMERCIAL DR PFLUGERVILLE, TX 78660  is complete
Selam International Mart & Cafe 10009 N LAMAR BLVD AUSTIN, TX 78753  is complete
Lupe Tortilla Mexican Restaurant 10515 N M

La Cocina De Consuelo 4516 BURNET RD AUSTIN, TX 78756  is complete
PF - Rollie Rolls 1101 W PECAN ST PFLUGERVILLE, TX 78660  is complete
WL - P. Terry's Burger Stand 701 S CAPITAL OF TEXAS HWY NB UNIT H WEST LAKE HILLS TX 78746  is complete
Moreliana Meat Market 5405 S PLEASANT VALLEY RD AUSTIN, TX 78744  is complete
Metcalf BBQ 1621 W 5TH ST AUSTIN, TX 78703  is complete
2 N 1 Salad Bar And Grill 901 W BRAKER LN AUSTIN, TX 78758  is complete
Hoffbrau 613 W 6TH ST AUSTIN, TX 78701  is complete
CVS Pharmacy  1701 E PARMER LN AUSTIN, TX 78754  is complete
Eastside High School 901 NEAL ST UNIT A AUSTIN, TX 78702  is complete
AA Foodmart #2 6649 MANOR RD AUSTIN, TX 78723  is complete
BBG's 214 E 6TH ST AUSTIN, TX 78701  is complete
WL - Tacodeli V, L.L.C. 701 S CAPITAL OF TEXAS HWY NB UNIT E-590 WEST LAKE HILLS TX 78746  is complete
Choo Sando 5222 BURNET RD AUSTIN, TX 78756  is complete
Jacoby's Austin - Catering Kitchen 3235 E CESAR CHAVEZ ST AUSTIN, TX 78702  is complete
Galaxy Cafe 100

Crave Catering 14611 BURNET RD AUSTIN, TX 78728  is complete
Tropical Island 14735 BRATTON LN AUSTIN, TX 78728  is complete
Torchy's Tacos 3005 S LAMAR BLVD AUSTIN, TX 78704  is complete
Cabo Bob's 500 E BEN WHITE BLVD AUSTIN, TX 78704  is complete
Hot Mama's Cafe 2401 E 6TH ST AUSTIN, TX 78702  is complete
Zoe's Safe Place 4606 CONNELLY ST AUSTIN, TX 78751  is complete
McNeil High School 5720 MC NEIL DR AUSTIN, TX 78729  is complete
Starbuck's Coffee # 10757 500 E BEN WHITE BLVD AUSTIN, TX 78704  is complete
BC - Whole Foods Market - Seafood 12601 HILL COUNTRY BLVD BEE CAVE, TX 78738  is complete
SV - La Madeleine # 202 5493 BRODIE LN SUNSET VALLEY, TX 78745  is complete
Kebab Shop, The 1201 BARBARA JORDAN BLVD AUSTIN, TX 78723  is complete
BC - Whole Foods Market - Bakery 12601 HILL COUNTRY BLVD BEE CAVE, TX 78738  is complete
BC - Walgreens  12200 BEE CAVE PKWY BEE CAVE, TX 78738  is complete
Steiner Ranch Elementary School 4001 N QUINLAN PARK RD AUSTIN, TX 78732  is complete
Frosty

Mi -Tienda 8007 GESSNER DR AUSTIN, TX 78753  is complete
Betsy's Billiards 1901 W WILLIAM CANNON DR AUSTIN, TX 78745  is complete
MN - Dollar General  12750 LAPOYNOR ST MANOR, TX 78653  is complete
ABIA - Barton Springs East/Ruta Maya 3600 PRESIDENTIAL BLVD AUSTIN, TX 78719  is complete
Rice Bowl Cafe 11220 N LAMAR BLVD AUSTIN, TX 78753  is complete
El Pollo Rico #6 1945 E OLTORF ST AUSTIN, TX 78741  is complete
Aura Hookah Lounge 2621 JONES RD AUSTIN, TX 78745  is complete
Residence Inn 6000 W US 290 HWY AUSTIN, TX 78735  is complete
Capriotti's Sandwich Shop 1200 BARBARA JORDAN BLVD AUSTIN, TX 78723  is complete
East Austin College Prep 5800 E MARTIN LUTHER KING JR BLVD AUSTIN, TX 78723  is complete
Tarka Indian Kitchen 2525 W ANDERSON LN AUSTIN, TX 78757  is complete
PF - Oriental Kitchen Restaurant 1100 GRAND AVENUE PKWY PFLUGERVILLE, TX 78664  is complete
Taqueria Morelos 3100 S US 183 HWY AUSTIN, TX 78744  is complete
PF - In - N - Out  1600 TOWN CENTER DR PFLUGERVILLE, TX 78660 

...

Wag A Bag 1701 GRAND AVENUE PKWY AUSTIN, TX 78660  is complete
Austin Artisan, The 2823 HANCOCK DR AUSTIN, TX 78731  is complete
Huckleberry Hospitality 1606 W STASSNEY LN AUSTIN, TX 78745  is complete
High Road on Dawson, The 700 DAWSON RD AUSTIN, TX 78704  is complete
Subway 3710 CRAWFORD AVE AUSTIN, TX 78731  is complete
7-Eleven Convenience Store  13641 N IH AUSTIN, TX 78753 is complete
Vida Pura Juicery 2610 MANOR RD AUSTIN, TX 78722  is complete
OOB - Sweet Briar Child Development Center 4600 NUCKOLS CROSSING RD AUSTIN, TX 78744  is complete
OOB - Stop N Start 6010 W PARMER LN AUSTIN, TX 78727  is complete
Dominos 2013 W WELLS BRANCH PKWY AUSTIN, TX 78728  is complete
Alamo Drafthouse Cinema 14028 N US 183 HWY SVRD SB BLDG F AUSTIN, TX 78717  is complete
Samsung Kitchen 3900 N CAPITAL OF TEXAS HWY AUSTIN, TX 78746  is complete
OOB - Midnight Cowboy 313 E 6TH ST AUSTIN, TX 78701  is complete
Palm Square Community Center 100 N IH AUSTIN, TX 78701 is complete
HEB Food Store  - Hwy 7

OOB - Michi Ramen Anderson Mill 10700 ANDERSON MILL RD AUSTIN, TX 78750  is complete
Chuck E. Cheese's 586 10515 N MOPAC EXPY AUSTIN, TX 78759  is complete
OOB - Sonic 155 W SLAUGHTER LN AUSTIN, TX 78748  is complete
Burger Fi 5701 W SLAUGHTER LN AUSTIN, TX 78749  is complete
Hacienda Jalisco 8766 RESEARCH BLVD AUSTIN, TX 78757  is complete
OOB - Moonlight Hookah Cafe 12636 RESEARCH BLVD AUSTIN, TX 78759  is complete
Emerald Point Store 5973 HI LINE RD AUSTIN, TX 78734  is complete
Reserve at Lake Travis Swim Park 2208 SEABISCUIT CV LAGO VISTA, TX 78669  is complete
PF - Wag A Bag  419 W PFLUGERVILLE PKWY PFLUGERVILLE, TX 78660  is complete
Wag-A-Bag 6294 MC NEIL DR AUSTIN, TX 78729  is complete
PF - Bento's Fit Kitchen 16708 PICADILLY CT ROUND ROCK, TX 78664  is complete
Domino's 5811 BERKMAN DR AUSTIN, TX 78723  is complete
Sonic 1637 W WELLS BRANCH PKWY AUSTIN, TX 78728  is complete
Cupprimo Cupcakery LLC 8650 SPICEWOOD SPRINGS RD AUSTIN, TX 78759  is complete
Dr. Wok 13201 POND SPR

Woko Loco 1201 BARBARA JORDAN BLVD AUSTIN, TX 78723  is complete
Hyatt Place North Central 7522 N IH AUSTIN, TX 78752 is complete
RW - Nothing Bundt Cakes 2805 BEE CAVES RD ROLLINGWOOD, TX 78746  is complete
OOB WL - Gold's Gym 701 S CAPITAL OF TEXAS HWY NB WEST LAKE HILLS TX 78746  is complete
Bamboo Garden 625 W BEN WHITE BLVD SVRD EB AUSTIN, TX 78745  is complete
OOB - Hookah Buzz 11657 RESEARCH BLVD AUSTIN, TX 78759  is complete
OOB Junior's King Daddy Ice 22900 WINDY VALLEY RD LEANDER, TX 78641  is complete
Tomunchi Frozen Yogurt 1701 W PARMER LN AUSTIN, TX 78727  is complete
El Chilito Tacos y Cafe 1623 E 7TH ST AUSTIN, TX 78702  is complete
OOB - Happy Chicks 360 NUECES ST AUSTIN, TX 78701  is complete
OOB - Nate's Baked Goods & Coffee 401 ORCHARD ST AUSTIN, TX 78703  is complete
OOB - Spirited Food Co. 2823 HANCOCK DR AUSTIN, TX 78731  is complete
MN - Sonic 11215 E US 290 HWY MANOR, TX 78653  is complete
Rack Daddy 1779 W WELLS BRANCH PKWY AUSTIN, TX 78728  is complete
Post Ro

Papa John's Pizza 5403 CAMERON RD AUSTIN, TX 78723  is complete
SV - Zoes Kitchen 5601 BRODIE LN SUNSET VALLEY, TX 78745  is complete
Rita's Mexican Restaurant 1934 RUTLAND DR AUSTIN, TX 78758  is complete
Subway Restaurant  3005 S LAMAR BLVD AUSTIN, TX 78704  is complete
Mom's Mandu 1023 SPRINGDALE RD AUSTIN, TX 78721  is complete
OOB Sonic Drive In 7708 BURNET RD AUSTIN, TX 78757  is complete
Ned Granger Building Cafe 314 W 11TH ST AUSTIN, TX 78701  is complete
OOB Mama Mia Italian Restaurant 1075 SPRINGDALE RD AUSTIN, TX 78721  is complete
Soup Peddler Real Food & Juice Bar 205 W 3RD ST AUSTIN, TX 78701  is complete
OOB - Frozen Rolls Creamery 704 N LAMAR BLVD AUSTIN, TX 78703  is complete
OOB-Subway 500 CANYON RIDGE DR AUSTIN, TX 78753  is complete
Crafty Loaf LLC 1934 RUTLAND DR AUSTIN, TX 78758  is complete
OOB - Zotas Night Club 4700 BURLESON RD AUSTIN, TX 78744  is complete
OOB East Austin Culinary Studio 2100 ROBERT BROWNING ST AUSTIN, TX 78723  is complete
360 Pizza, LLC 6203

TypeError:  'NoneType' object is not subscriptable Classic Cooking & Catering 1124 S IH AUSTIN, TX 78704
Austinuts 2900 W ANDERSON LN AUSTIN, TX 78757  is complete
Seventh Flag Coffee 1506 S 1ST ST AUSTIN, TX 78704  is complete
Drink.Well. 207 E 53RD ST AUSTIN, TX 78751  is complete
Alamo Drafthouse Cinema 2700 W ANDERSON LN AUSTIN, TX 78757  is complete
Scoreboard Sports Bar & Grill 6507 JESTER BLVD AUSTIN, TX 78750  is complete
Monitos Mexican Grill & Cantina 8127 MESA DR AUSTIN, TX 78759  is complete
Gattis Pizza 2931 W ANDERSON LN AUSTIN, TX 78757  is complete
OOB - Fast Market # 8186 110 W SLAUGHTER LN AUSTIN, TX 78748  is complete
Peace of Mind 2825 HANCOCK DR AUSTIN, TX 78731  is complete
Whoa BBQ 5530 BURNET RD AUSTIN, TX 78756  is complete
OOB Ni-Kome 111 CONGRESS AVE AUSTIN, TX 78701  is complete
Brooke Elementary 3100 E 4TH ST AUSTIN, TX 78702  is complete
The Steeping Room 9715 BURNET RD AUSTIN, TX 78758  is complete
Dai Due Taqueria 111 CONGRESS AVE AUSTIN, TX 78701  is co

In [13]:
df['new_latitude'] = latitude_list
df['new_longitude'] = longitude_list

In [23]:
df.to_csv('data/inspection-scores.csv', index=False)

## Export Dataframe with All Scores

In [10]:
all_scores.drop(['coordinates', 'original_latitude', 'original_longitude'], axis=1, inplace=True)

In [11]:
all_scores.to_csv('data/all_scores.csv', index=False)